In [1]:
import pandas as pd

RMPBothell = pd.read_csv('ratemyprof.csv')
RMPSeattle = pd.read_csv('ratemyprofseattle.csv')

## Seattle exactly the same
RMPBothell.head()

,web-scraper-order,web-scraper-start-url,more-button,professor link,professor link-href,Review-Body,Date,Course-Name,For-Credit,Attendance,Take-Again,Grade,Textbook,Quality,Difficulty,more-ratings
0,1714778254-1,https://www.ratemyprofessors.com/search/profes...,NaN,QUALITY2.01 ratingsLorraine YuBusinessUniversi...,https://www.ratemyprofessors.com/professor/198...,"It's not hard class, but don't expect the 4.0....","Mar 8th, 2015",BUS340,For Credit: Yes,Textbook: Yes,NaN,NaN,NaN,Quality2.0,2.0,NaN
1,1714778261-2,https://www.ratemyprofessors.com/search/profes...,NaN,QUALITY2.51 ratingsKarrin KlotzLawUniversity o...,https://www.ratemyprofessors.com/professor/198...,Syllabus was clear on what the daily homework ...,"Feb 21st, 2016",BCUSP202,For Credit: Yes,Textbook: Yes,NaN,NaN,NaN,Quality2.5,3.0,NaN
2,1714778284-3,https://www.ratemyprofessors.com/search/profes...,NaN,QUALITY4.87 ratingsJohn BridgeEngineeringUnive...,https://www.ratemyprofessors.com/professor/198...,Bridge is the coolest prof I've ever had. His ...,"Nov 20th, 2023",BME221,For Credit: Yes,Attendance: Not Mandatory,Would Take Again: Yes,Would Take Again: Yes,Textbook: Yes,Quality5.0,2.0,NaN
3,1714778284-4,https://www.ratemyprofessors.com/search/profes...,NaN,QUALITY4.87 ratingsJohn BridgeEngineeringUnive...,https://www.ratemyprofessors.com/professor/198...,Bridge makes it abundantly clear what you need...,"Aug 22nd, 2020",ME341,For Credit: Yes,Attendance: Not Mandatory,Would Take Again: Yes,Would Take Again: Yes,Textbook: Yes,Quality5.0,3.0,NaN
4,1714778284-5,https://www.ratemyprofessors.com/search/profes...,NaN,QUALITY4.87 ratingsJohn BridgeEngineeringUnive...,https://www.ratemyprofessors.com/professor/198...,One of the best professors I have ever had. Tr...,"Apr 22nd, 2020",BENGR321,For Credit: Yes,Attendance: Mandatory,Would Take Again: Yes,Would Take Again: Yes,Textbook: Yes,Quality5.0,3.0,NaN


In [2]:
## Unnecessary columns from web scraping tool
RMPBothell.drop(columns=['web-scraper-order', 
                         'web-scraper-start-url', 
                         'more-button', 
                         'professor link', 
                         'professor link-href', 
                         'more-ratings'], inplace=True)
RMPSeattle.drop(columns=['web-scraper-order', 
                         'web-scraper-start-url', 
                         'more-button', 
                         'professor link', 
                         'professor link-href', 
                         'more-ratings'], inplace=True)

In [3]:
combined_RMP = pd.concat([RMPBothell, RMPSeattle], ignore_index=True)

In [4]:
# Clean 'Quality' column
combined_RMP['Quality'] = combined_RMP['Quality'].str.split('Quality').str[-1]

# Initialize empty lists for each column
for_credit_list = []
attendance_list = []
take_again_list = []
grade_list = []
textbook_list = []

# Iterate over each row in the DataFrame
for index, row in combined_RMP[['For-Credit', 'Attendance', 'Take-Again', 'Grade', 'Textbook']].iterrows():
    # Initialize default values as None
    for_credit = attendance = take_again = grade = textbook = None
    # Split each value in the row based on ':'
    for item in row:
        if pd.notna(item) and ':' in item:
            if pd.notna(item):
                key, value = item.split(': ', 1)
                # Assign values to appropriate variables based on the key
                if key == 'For Credit':
                    for_credit = value
                elif key == 'Attendance':
                    attendance = value
                elif key == 'Would Take Again':
                    take_again = value
                elif key == 'Grade':
                    grade = value
                elif key == 'Textbook':
                    textbook = value
    # Append values to appropriate lists
    for_credit_list.append(for_credit)
    attendance_list.append(attendance)
    take_again_list.append(take_again)
    grade_list.append(grade)
    textbook_list.append(textbook)

cleaned_data = {
    'For-Credit': for_credit_list,
    'Attendance': attendance_list,
    'Take-Again': take_again_list,
    'Grade': grade_list,
    'Textbook': textbook_list
    }

cleaned_df = pd.DataFrame(cleaned_data)
combined_RMP['For-Credit'] = cleaned_df['For-Credit']
combined_RMP['Attendance'] = cleaned_df['Attendance']
combined_RMP['Take-Again'] = cleaned_df['Take-Again']
combined_RMP['Grade'] = cleaned_df['Grade']
combined_RMP['Textbook'] = cleaned_df['Textbook']


combined_RMP.head()


,Review-Body,Date,Course-Name,For-Credit,Attendance,Take-Again,Grade,Textbook,Quality,Difficulty
0,"It's not hard class, but don't expect the 4.0....","Mar 8th, 2015",BUS340,Yes,None,None,None,Yes,2.0,2.0
1,Syllabus was clear on what the daily homework ...,"Feb 21st, 2016",BCUSP202,Yes,None,None,None,Yes,2.5,3.0
2,Bridge is the coolest prof I've ever had. His ...,"Nov 20th, 2023",BME221,Yes,Not Mandatory,Yes,None,Yes,5.0,2.0
3,Bridge makes it abundantly clear what you need...,"Aug 22nd, 2020",ME341,Yes,Not Mandatory,Yes,None,Yes,5.0,3.0
4,One of the best professors I have ever had. Tr...,"Apr 22nd, 2020",BENGR321,Yes,Mandatory,Yes,None,Yes,5.0,3.0


In [5]:
# extract course number to determine upper or lower class
combined_RMP['Course-Number'] = combined_RMP['Course-Name'].str.extract(r'(\d+)')
combined_RMP['Course-Name'] = combined_RMP['Course-Name'].str.extract(r'([A-Za-z]+)')



In [6]:
# format Date column
def remove_ordinal(date_str):
    return date_str.replace('th', '').replace('st', '').replace('nd', '').replace('rd', '')

combined_RMP['Date'] = pd.to_datetime(combined_RMP['Date'].apply(remove_ordinal), format='%b %d, %Y')

In [7]:
# re-order to keep course name and number together and keep main targets on the right.
combined_clean_RMP = combined_RMP[['Review-Body', 'Date', 'Course-Name', 'Course-Number', 'For-Credit', 'Attendance', 'Take-Again', 'Grade', 'Textbook', 'Quality', 'Difficulty']]
combined_clean_RMP.head()

,Review-Body,Date,Course-Name,Course-Number,For-Credit,Attendance,Take-Again,Grade,Textbook,Quality,Difficulty
0,"It's not hard class, but don't expect the 4.0....",2015-03-08,BUS,340,Yes,None,None,None,Yes,2.0,2.0
1,Syllabus was clear on what the daily homework ...,2016-02-21,BCUSP,202,Yes,None,None,None,Yes,2.5,3.0
2,Bridge is the coolest prof I've ever had. His ...,2023-11-20,BME,221,Yes,Not Mandatory,Yes,None,Yes,5.0,2.0
3,Bridge makes it abundantly clear what you need...,2020-08-22,ME,341,Yes,Not Mandatory,Yes,None,Yes,5.0,3.0
4,One of the best professors I have ever had. Tr...,2020-04-22,BENGR,321,Yes,Mandatory,Yes,None,Yes,5.0,3.0


In [8]:
combined_clean_RMP.to_csv('Cleaned_UW_RMP.csv', index=False)